### 테스트용 데이터 수집 시 사용한 코드
- 모델 학습 중 eval에 사용된 데이터는 학습 시 사용했던 데이터와 비교적 유사한 데이터들로 구성되어 있어, 정확도의 신뢰도가 낮았음 (학습 데이터는 소수의 모집단에서 추출되었으므로)
- 모델을 재차 검증하기 위해 indj 직원들을 손 제스처를 수집하여 test 하고자함
- test 시 사용될 데이터를 수집하기 위해 실시간 웹캠을 사용하였음
- 실시간 웹캠을 통해 수집된 손의 랜드마크값을 csv 로 저장하여 test.ipynb 에서 test 하였음

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import sys


print("Python version:", sys.version)
print("cv2 version:", cv2.__version__)
print("mediapipe version", mp.__version__)
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)


2025-06-18 15:47:14.766286: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-18 15:47:14.772811: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-18 15:47:14.831834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-18 15:47:14.831896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-18 15:47:14.837760: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Python version: 3.10.16 (main, Dec 11 2024, 16:24:50) [GCC 11.2.0]
cv2 version: 4.11.0
mediapipe version 0.10.21
pandas version: 2.2.3
numpy version: 1.26.3


In [ ]:
# 수집할 제스처 클래스를 지정 (추후 파일명에 사용, 분류 목적)
# 다른 제스처를 수집할 때는 무조건 class_num을 수정해줘야 정확한 수집이 가능
class_num=10

# MediaPipe Hand 모델을 통해 detection 하는 손의 개수 (일반적으로 1개만 가능)
max_num_hands=1


gesture={
    0 : 'Right',
    1 : 'Left',
    2 : 'Turn Clockwise',
    3 : 'Turn Anticlockwise',
    4 : 'Twinkle',
    5 : 'Okay',
    6 : 'Stop',
    7 : 'Rock',
    8 : 'Play',
    9 : 'Slide',
    10 : 'none'
}

In [ ]:
# Mediapipe hand 초기화 / 로드

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# MediaPipe hand 모델 설정
hands = mp_hands.Hands(
    max_num_hands=max_num_hands,
    # 손을 detection 할 때 신뢰도 0.5가 넘으면 detection / 올릴수록 더 깐깐하게 검출
    min_detection_confidence=0.5,
    # tracking 신뢰도 / 올릴수록 더 깐깐하게 tracking
    min_tracking_confidence=0.5)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1750229244.311846  782028 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1750229244.324164  782028 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
# make Data set code

# 만약 기존의 데이터 파일에서 붙여서 계속 데이터를 모으고 싶다면, 아래 파일명만 바꾸면 됨.
# default_array = np.genfromtxt('./data/start_file.csv', delimiter=',')

# 기존의 데이터 없이 데이터 수집하는 경우.
default_array=np.array(range(100),dtype='float64')


VIDEO_URL = 'http://192.168.0.103:5000/video'
webcam = cv2.VideoCapture(VIDEO_URL)

# 약 1.5초 분량 프레임 수 (FPS가 30이라면 1.5초 ≈ 45프레임)
target_frames = 1800  # 60초 기준 -> 1800 프레임 수집
collected_frames = 0

while webcam.isOpened():  # webcap 열려있는지 확인.
    seq=[]
    status, frame =webcam.read()

    if not status:
        continue

    frame=cv2.flip(frame,1)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:

        # 손에 대한 정보 실시간(프레임 느낌)으로 계속 가져옴.
        # res 에 21개의 (x,y,z) 정보가 들어있음( 각각 관절의 위치 값들)
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 4))

            
            # 21개의 관절 포인트에 대해 joint 할당함.
            for j, lm in enumerate(res.landmark):
                
                # x ,y, z(distance between camera and hand)
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility] # [21,4]
            

            # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
            v = v2 - v1 # [20,4]
            
            
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]   #[20,4]
             

            # Get angle using arcos of dot product
            # angle [15,1]
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) 
            


            angle = np.degrees(angle) # Convert radian to degree
            angle = np.array([angle], dtype=np.float32)
            
            #  'angle + class_num'
            #   angle_label [16,1]
            angle_label = np.append(angle, class_num) 
           
            # joint[21,4] + angle_label[16,1] 
            # joint_angle_label [100,]
            joint_angle_label = np.concatenate([joint.flatten(), angle_label])



            mp_drawing.draw_landmarks(frame, res, mp_hands.HAND_CONNECTIONS)

            

            # np.array() 형태로 계속 붙음.(넘파이 형태임.)
            seq.append(joint_angle_label)  
            

        data=np.array(seq)
        default_array = np.vstack((default_array, data))
        collected_frames += 1
        print(f"Collected frame: {collected_frames}")  # 실시간 출력
        
        # print('file shape : ',default_array.shape)

            
    
    cv2.imshow('Dataset', frame)
    # 지정한 프레임(초(sec)) 분량 수집 후 자동 종료
    if collected_frames >= target_frames:
        print(f"{collected_frames} frames collected. Saving file...")
        break
    
    if cv2.waitKey(1) == ord('q'):
        break


# save file
filename = f'../data_gesture_{gesture[class_num]}_1.csv'
np.savetxt('../data_gesture_'+gesture[class_num]+'.csv', default_array[1:,:], delimiter=',')
print(f"Saved to: {filename}")

webcam.release()
cv2.destroyAllWindows()

Collected frame: 1
Collected frame: 2
Collected frame: 3
Collected frame: 4
Collected frame: 5
Collected frame: 6
Collected frame: 7
Collected frame: 8
Collected frame: 9
Collected frame: 10
Collected frame: 11
Collected frame: 12
Collected frame: 13
Collected frame: 14
Collected frame: 15
Collected frame: 16
Collected frame: 17
Collected frame: 18
Collected frame: 19
Collected frame: 20
Collected frame: 21
Collected frame: 22
Collected frame: 23
Collected frame: 24
Collected frame: 25
Collected frame: 26
Collected frame: 27
Collected frame: 28
Collected frame: 29
Collected frame: 30
Collected frame: 31
Collected frame: 32
Collected frame: 33
Collected frame: 34
Collected frame: 35
Collected frame: 36
Collected frame: 37
Collected frame: 38
Collected frame: 39
Collected frame: 40
Collected frame: 41
Collected frame: 42
Collected frame: 43
Collected frame: 44
Collected frame: 45
Collected frame: 46
Collected frame: 47
Collected frame: 48
Collected frame: 49
Collected frame: 50
Collected

In [17]:
# make Data set code

# 만약 기존의 데이터 파일에서 붙여서 계속 데이터를 모으고 싶다면, 아래 파일명만 바꾸면 됨.
# default_array = np.genfromtxt('./data/start_file.csv', delimiter=',')

# 기존의 데이터 없이 데이터 수집하는 경우.
default_array=np.array(range(100),dtype='float64')


VIDEO_URL = 'http://192.168.0.103:5000/video'
webcam = cv2.VideoCapture(VIDEO_URL)  # 몇번째 카메라인지 설정.

# 약 1.5초 분량 프레임 수 (FPS가 30이라면 1.5초 ≈ 45프레임)
target_frames = 900
collecting = False  # 수집 트리거
collected_frames = 0

while webcam.isOpened():  # webcap 열려있는지 확인.
    seq=[]
    status, frame =webcam.read()

    if not status:
        continue

    frame=cv2.flip(frame,1)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:

        # 손에 대한 정보 실시간(프레임 느낌)으로 계속 가져옴.
        # res 에 21개의 (x,y,z) 정보가 들어있음( 각각 관절의 위치 값들)
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 4))

            
            # 21개의 관절 포인트에 대해 joint 할당함.
            for j, lm in enumerate(res.landmark):
                
                # x ,y, z(distance between camera and hand)
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility] # [21,4]
            

            # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
            v = v2 - v1 # [20,4]
            
            
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]   #[20,4]
             

            # Get angle using arcos of dot product
            # angle [15,1]
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) 
            


            angle = np.degrees(angle) # Convert radian to degree
            angle = np.array([angle], dtype=np.float32)
            
            #  'angle + class_num'
            #   angle_label [16,1]
            angle_label = np.append(angle, class_num) 
           
            # joint[21,4] + angle_label[16,1] 
            # joint_angle_label [100,]
            joint_angle_label = np.concatenate([joint.flatten(), angle_label])



            mp_drawing.draw_landmarks(frame, res, mp_hands.HAND_CONNECTIONS)

            

            # np.array() 형태로 계속 붙음.(넘파이 형태임.)
            seq.append(joint_angle_label)  
            

        data=np.array(seq)
        if collecting:
            default_array = np.vstack((default_array, data))
            collected_frames += 1
            print(f"Collected frame: {collected_frames}")  # 실시간 출력
        
        # print('file shape : ',default_array.shape)

            
    
    cv2.imshow('Dataset', frame)
    # 1.5초 분량 수집 후 자동 종료
    if collected_frames >= target_frames:
        print(f"{collected_frames} frames collected. Saving file...")
        break
    
    key = cv2.waitKey(1)
    
    if key == ord('q'):
        break
    elif key == 32 and not collecting:  # space key
        print("▶ 스페이스바 누름 → 수집 시작")
        collecting = True
        collected_frames = 0
 

# save file
filename = f'../test/data_gesture_{gesture[class_num]}_1.csv'
np.savetxt('../test/data_gesture_'+gesture[class_num]+'.csv', default_array[1:,:], delimiter=',')
print(f"Saved to: {filename}")

webcam.release()
cv2.destroyAllWindows()

▶ 스페이스바 누름 → 수집 시작
Collected frame: 1
Collected frame: 2
Collected frame: 3
Collected frame: 4
Collected frame: 5
Collected frame: 6
Collected frame: 7
Collected frame: 8
Collected frame: 9
Collected frame: 10
Collected frame: 11
Collected frame: 12
Collected frame: 13
Collected frame: 14
Collected frame: 15
Collected frame: 16
Collected frame: 17
Collected frame: 18
Collected frame: 19
Collected frame: 20
Collected frame: 21
Collected frame: 22
Collected frame: 23
Collected frame: 24
Collected frame: 25
Collected frame: 26
Collected frame: 27
Collected frame: 28
Collected frame: 29
Collected frame: 30
Collected frame: 31
Collected frame: 32
Collected frame: 33
Collected frame: 34
Collected frame: 35
Collected frame: 36
Collected frame: 37
Collected frame: 38
Collected frame: 39
Collected frame: 40
Collected frame: 41
Collected frame: 42
Collected frame: 43
Collected frame: 44
Collected frame: 45
Collected frame: 46
Collected frame: 47
Collected frame: 48
Collected frame: 49
Collected 